In [1]:
import plasma.search_engines as ses
import pandas as pd
import plasma.logging as plogging

/u03/thanhhm/envs/research/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_feather('/u02/thanhhm/xproject/as/databases/icd10.feather')
engine = ses.GraphMatcher(
    df['name'], 
    group_splitter='([^:\n;?!.()]+)',
    top_k=5
)

In [3]:
from plasma.search_engines.path_walker_2 import PathWalker

In [4]:
old_walker = engine.path_walker
new_path_walker = PathWalker(engine.path_walker.threshold, engine._graph, engine._data)

In [5]:
query = 'Tổn thương nông tác động nhiều vùng cơ thể (Đa chấn thương, vết thương cẳng chân trái do tai nạn giao thông)'

In [7]:
with plogging.Timer(print) as timer:
    new_results = engine.run(query)
timer.duration

2.2401976585388184

In [12]:
query[64:80]

'thương cẳng chân'

In [11]:
new_results

data_index  \
query_start_index query_end_index               
0                 42                    24754   
44                58                    24919   
                  58                    24920   
52                63                    22517   
60                70                    23325   
                  70                    23339   
                  70                    23397   
                  70                    23346   
                  70                    23353   
64                80                    24424   
                  80                    24361   
                  80                    24362   
                  80                    24441   
                  80                    24442   
86                107                   28554   
                  107                   28579   
                  107                   28603   
                  107                   28550   
                  107                   28577   

                                                                       original_text  \
query_start_index query_end_index                                                      
0                 42                      Tổn thương nông tác động nhiều vùng cơ thể   
44                58                            Đa chấn thương cắt cụt chưa xác định   
                  58                           Đa chấn thương đứt lìa không xác định   
52                63               Tổn thương kết mạc và xước giác mạc không có d...   
60                70                                                 Vết thương lách   
                  70                                                  Vết thương tụy   
                  70                                                 Vết thương thận   
                  70                                               Vết thương dạ dày   
                  70                                             Vết thương ruột non   
64                80                                Gãy nhiều vị trí xương cẳng chân   
                  80                            Gãy xương cẳng chân, bao gồm cổ chân   
                  80                            Gẫy xương cẳng chân, bao gồm cổ chân   
                  80                         Gãy xương cẳng chân, phần chưa xác định   
                  80                        Gãy xương cẳng chân, phần không xác định   
86                107              Người đi xe công nghiệp bị thương không phải d...   
                  107              Người đi xe nông nghiệp bị thương không phải d...   
                  107              Người đi xe xây dựng bị thương không phải do t...   
                  107              Lái xe của xe công nghiệp bị thương không phải...   
                  107              Lái xe của xe nông nghiệp bị thương không phải...   

                                   substring_matching_score  \
query_start_index query_end_index                             
0                 42                               1.000000   
44                58                               1.000000   
                  58                               1.000000   
52                63                               0.800000   
60                70                               1.000000   
                  70                               1.000000   
                  70                               1.000000   
                  70                               1.000000   
                  70                               1.000000   
64                80                               0.888889   
                  80                               0.888889   
                  80                               0.888889   
                  80                               0.888889   
                  80                               0.888889   
86                107                              1.000000   
                  107                              1.000000   
                  107 